In [1]:
using Agents, Turing, Random

In [2]:
@agent Depozitar OSMAgent begin
    capacitate_m3::Float32
    greutate_max_g::Float32
    avg_timp_contact_s::Float32
    dev_timp_contact_s::Float32
    timpi_contact_s::Vector{Float32}
    probabilitate_livrare_corecta::Float32
    comision_prc::Float32
end

In [3]:
@agent Transportor OSMAgent begin
    capacitate_m3::Float32
    distanta_max_km::Float32
    tol_distanta_max::Float32
    istoric_distante::Vector{Float32}
    avg_timp_contact_s::Float32
    dev_timp_contact_s::Float32
    dev_timp_prezis_sosire::Float32
    timpi_prezisi_sosire::Vector{Float32}
    timpi_contact_s::Vector{Float32}
    probabilitate_livrare_corecta::Float32
    comision_prc::Float32
end

In [4]:
function initialise(; map_path = "map.osm")
    model = ABM(Union{Transportor, Depozitar}, OpenStreetMapSpace(map_path))
    for id in 1:100
        start = random_position(model)
        if model.rng > .8
            finish = start
            e_depozitar = true
            e_transportor = false
            capacitate_m3 = rand(Exponential(20))
            timp_iesire_bloc_s = rand(Exponential(100))
        else
            finish = random_position(model)
            e_depozitar = false
            e_transportor = true
            capacitate_m3 = rand(Exponential(2))
            timp_iesire_bloc_s = 0.
        end
        route = OSM.plan_route(start, finish, model)
        muncitor = Lucrator(id, start, route, finish, e_depozitar, e_transportor, capacitate_m3, timp_iesire_bloc_s)
        add_agent_pos!(human, model)
    end
    return model
end

initialise (generic function with 1 method)

In [5]:
m = initialise()

┌ Warning: AgentType is not concrete. If your agent is parametrically typed, you're probably seeing this warning because you gave `Agent` instead of `Agent{Float64}` (for example) to this function. You can also create an instance of your agent and pass it to this function. If you want to use `Union` types for mixed agent models, you can silence this warning.
└ @ Agents /home/drakop/.julia/packages/Agents/HGute/src/core/model.jl:297


LoadError: MethodError: no method matching isless(::Float64, ::MersenneTwister)
[0mClosest candidates are:
[0m  isless(::Union{Number, Dates.AbstractTime}, [91m::IntervalSets.ClosedInterval{T} where T[39m) at /home/drakop/.julia/packages/AxisArrays/FWWEV/src/intervals.jl:53
[0m  isless(::AbstractFloat, [91m::Union{StatsBase.PValue, StatsBase.TestStat}[39m) at /home/drakop/.julia/packages/StatsBase/DWW4r/src/statmodels.jl:524
[0m  isless(::AbstractFloat, [91m::ForwardDiff.Dual{Ty, V, N} where {V, N}[39m) where Ty at /home/drakop/.julia/packages/ForwardDiff/UDrkY/src/dual.jl:145
[0m  ...

In [ ]:
function agent_step!(agent, model)
    move_along_route!(agent, model, 25)
    if is_stationary(agent, model) && rand(model.rng) < .1
        OSM.random_route!(agent, model)
        move_along_route!(agent, model, 25)
    end
    
    if agent.infected
        map(i -> model[i].infected = true, nearby_ids(agent, model, 50))
    end
end

In [ ]:
using  OpenStreetMapXPlot
using Plots
gr()

In [ ]:
ac(agent) = agent.infected ? :green : :black

as(agent) = agent.infected ? 6 : 5

function plotagents(model)
    ids = model.scheduler(model)
    colors = [ac(model[i]) for i in ids]
    sizes = [as(model[i]) for i in ids]
    markers = :circle
    pos = [OSM.map_coordinates(model[i], model) for i in ids]
    scatter!(
        pos;
        markercolor = colors,
        markersize = sizes,
        markershapes = markers,
        label = "",
        markerstrokewidth = .5,
        markerstrokecolor = :black,
        markeralpha = .7,
    )
end

In [ ]:
model = initialise()

In [ ]:
frames = @animate for i in 0:200
    i > 0 && step!(model, agent_step!, 1)
    plotmap(model.space.m)
    plotagents(model)
end
gif(frames, "outbreak.gif", fps = 15)